<a href="https://colab.research.google.com/github/PranavRathod399/Blacklace/blob/main/code/blacknet/V-2_symptom_triage_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import re

# -----------------------------------
# 1. Base risk scores for symptoms
# -----------------------------------
symptom_score = {
    "fever": 1,
    "cough": 1,
    "sore throat": 1,
    "shortness of breath": 2,
    "blood": 3,
    "chest pain": 3,
    "headache": 1,
    "nausea": 1,
    "vomiting": 2,
    "diarrhea": 2,
}

In [35]:
# -----------------------------------
# 2. Home care & escalation advice
# -----------------------------------
symptom_care = {
    "fever": {
        "home_care": "Stay hydrated, get rest, and use a cool damp cloth on your forehead.",
        "escalation": "If fever lasts more than 3 days or is very high, consult a doctor."
    },
    "cough": {
        "home_care": "Drink warm fluids and rest your voice.",
        "escalation": "If cough includes blood or lasts more than a week, seek medical attention."
    },
    "sore throat": {
        "home_care": "Gargle with warm salt water and stay hydrated.",
        "escalation": "If severe or lasts more than 3 days, consider seeing a doctor."
    },
    "nausea": {
        "home_care": "Eat light meals and drink clear fluids slowly.",
        "escalation": "If persistent or accompanied by vomiting, consult a doctor."
    },
    "chest pain": {
        "home_care": "If mild, rest and monitor your breathing.",
        "escalation": "Seek emergency care if severe, sharp, or spreads."
    }
}

In [36]:
# -----------------------------------
# 3. Symptom & duration extractor
# -----------------------------------
def extract_symptoms_and_duration(text):
    text = text.lower()
    symptoms_found = [symptom for symptom in symptom_score if symptom in text]

    patterns = [
        r"for (\d+ (day|days|week|weeks))",
        r"for (a few|few|several) (days|weeks)",
        r"since (yesterday|last night|monday|tuesday|wednesday|thursday|friday|saturday|sunday)",
        r"since (a few|few|several) (days|weeks)",
        r"(over|during) the past (\d+ (day|days|week|weeks))"
    ]

    duration = None
    for pattern in patterns:
        match = re.search(pattern, text)
        if match:
            duration = match.group(0)
            break

    return symptoms_found, duration

In [37]:
# -----------------------------------
# 4. Triage logic rules
# -----------------------------------
def assess_risk(symptoms, duration=None):
    total_score = sum(symptom_score.get(symptom, 0) for symptom in symptoms)

    if "cough" in symptoms and "blood" in symptoms:
        return "Emergency care advised"
    if "fever" in symptoms and "shortness of breath" in symptoms:
        return "Consult a doctor"

    if duration:
        if "week" in duration.lower() or "7 days" in duration.lower():
            total_score += 1

        if total_score >= 5:
            return "Consult a doctor"
        elif total_score >= 2:
            return "Monitor at home"
        elif total_score > 0:
            return "Mild: Home care"
        else:
            return "Insufficient data for triage"

In [53]:
# -----------------------------------
# 5. Care advice fetcher
# -----------------------------------
def get_care_advice(symptoms):
    advice = {}
    for symptom in symptoms:
        if symptom in symptom_care:
            advice[symptom] = symptom_care[symptom]
    return advice

In [55]:
# -----------------------------------
# 6. Full pipeline for analysis
# -----------------------------------
def analyze_input(text):
    symptoms, duration = extract_symptoms_and_duration(text)
    triage = assess_risk(symptoms, duration)

    if triage in ["Mild: Home care", "Monitor at home"]:
        care_advice = get_care_advice(symptoms)
    elif triage in ["Emergency care advised", "Consult a doctor"]:
        care_advice = {
            "emergency_action": "Call 911 or your local emergency number immediately."
        }
    else:
        care_advice = {}

    return {
        "symptoms": symptoms,
        "duration": duration,
        "triage": triage,
        "care_advice": care_advice
    }

In [59]:
# -----------------------------------
# 7. Optional CLI test
# -----------------------------------
if __name__ == "__main__":
    user_input = input("Describe your symptoms: ")
    result = analyze_input(user_input)

    print("\n--- Symptom Analysis ---")
    print("Symptoms Detected :", ", ".join(result["symptoms"]) if result["symptoms"] else "None")
    print("Duration           :", result["duration"] or "Not specified")
    print("Triage Assessment  :", result["triage"])

    if result["care_advice"]:
        print("\n--- Home Care Advice ---")
        # Check if the 'care_advice' is a dictionary with multiple symptoms
        if isinstance(result["care_advice"], dict) and len(result["care_advice"]) > 1:
            for symptom, advice in result["care_advice"].items():
                print(f"\n{symptom.title()}:")
                print(f"  Home Care   : {advice['home_care']}")
                print(f"  Escalation  : {advice['escalation']}")
        # If it's for emergency cases, print the emergency action
        elif "emergency_action" in result["care_advice"]:
            print(f"\n{result['care_advice']['emergency_action']}") # Print the emergency action directly

Describe your symptoms: I've got fever headache since 2 days today I got blood in my cough

--- Symptom Analysis ---
Symptoms Detected : fever, cough, blood, headache
Duration           : Not specified
Triage Assessment  : Emergency care advised

--- Home Care Advice ---

Call 911 or your local emergency number immediately.
